# Notebook with just the LNS functions:
Jessica Reichelt

### 0. Import, Informations and helping functions

In [30]:
## Import:
'''Installed packages:
    pandas
    geopandas
    pyshp
    networkx
    pyvis
    shapely
    openpyxl
'''
## Calculation:
import pandas as pd
import math
import random
import numpy as np
import plotly.express as px
import time
import timeit
from timeit import default_timer as timer
#from itertools import chain

## Graph:
import networkx as nx
from pyvis.network import Network
#from collections import deque

## Visualization:
import matplotlib.pyplot as plt
import shapely.geometry
import shapefile
import geopandas as gpd
from shapely.geometry import mapping, Polygon, Point, MultiPolygon
from IPython import display
central_europe = gpd.GeoDataFrame.from_file("intersection.shp")
i = np.argmax(central_europe.geometry.area)
europe_polygon = central_europe.geometry[i][0]

In [31]:
def calc_dist(lat_start, lat_end, lon_start, lon_end):
    # Input: latitude & longitude of two points (example below)
    # Output: (air-line distance in km and) estimated time difference in minutes
    ''' 
    CALCULATION OF DISTANCE IN MINUTES (ROUNDED) OF TWO COORDINATES
    with fixed speed and bee-line distance
    '''
    speed = 75  # estimated avg speed in km/h
    R = 6373.0
    lat2 = math.radians(lat_end)
    lat1 = math.radians(lat_start)
    lon2 = math.radians(lon_end)
    lon1 = math.radians(lon_start)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(
        dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return np.ceil(distance / speed * 60)

In [32]:
def bounds(routes, fixcosts=500):

    n = len(routes)
    try:
        km_costs_base = np.sum(np.array(routes)[:, -1], axis=0)[0]
    except:
        km_costs_base = sum([routes[i][-1][0] for i in range(n)])
    lb = int(n / 4) * fixcosts + km_costs_base  #we assume 1/4n drivers needed
    ub = n * fixcosts + km_costs_base
    return (lb, ub)

In [33]:
def print_requ(routes):
    '''
        prints only the requests of a solution (assemly of tours)
        for printing single tours use: print_requ([tour])
    '''
    print("\033[1m", "\n Requests only:", "\033[0m")
    for i in range(len(routes)):
        print(routes[i][4::4])
    return

In [34]:
def route_vis(requs):
    
    pick_starts = np.array(time_windows[0][0])
    del_ends = np.array(time_windows[1][1])
    picks = (pick_starts/60/24)[0][requs]
    dels = (del_ends/60/24)[0][requs]
    days = np.array(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
    
    fig, axs = plt.subplots()
    axs.set_aspect('equal', 'datalim') 
    xs, ys = europe_polygon.exterior.xy 
    k=0
    jet= plt.get_cmap('jet')
    colors = iter(jet(np.linspace(0,1,n)))
    axs.fill(xs, ys, alpha=0.5, ec='none')
    jet= plt.get_cmap('jet')
    colors = iter(jet(np.linspace(0,1,len(requs))))
    k=0
    for i in requs:
        x = lons[2*i]
        lon = lons[2*i+1]
        y = lats[2*i] 
        lat = lats[2*i+1]
        axs.scatter(x, y, color='g')
        axs.scatter(lon, lat, color='r')
        axs.plot([lon,x], [lat,y],label = 'r.'+str(i) ,color=next(colors))
        axs.annotate(days[int(picks[k])],([lon,lat]))
        axs.annotate(days[int(dels[k])],([x,y]))
        axs.legend()
        k+=1
    plt.show()
    return()

### 1. Automated Excel Data Reading:

In [35]:
def read_data(file, sheet):
    df = pd.read_excel(file, sheet_name=sheet)
    n = len(df.lat1)

    latitudes = []
    longitudes = []
    requests = list()
    Name = []
    pick_start = []
    pick_end = []
    delivery_start = []
    delivery_end = []
    i = 0
    for i in range(n):
        requests.append((2 * i, 2 * i + 1, i))  # Aufbau: (location #1, loc#2, request #)
        latitudes.append(df.lat1[i])
        latitudes.append(df.lat2[i])
        longitudes.append(df.lon1[i])
        longitudes.append(df.lon2[i])
        pick_start.append(df.pick_start[i])
        pick_end.append(df.pick_end[i])
        delivery_start.append(df.delivery_start[i])
        delivery_end.append(df.delivery_end[i])
        #Name.append(df.Ort1[i])
        #Name.append(df.Ort2[i])
    pick_start.append(0)
    pick_end.append(10080)
    delivery_start.append(0)
    delivery_end.append(10080)
    return (n, requests, latitudes, longitudes, pick_start, pick_end,
            delivery_start, delivery_end)

## 2. Labeling

### Resource Extension Functions (REF):
(Label changing Functions)

In [54]:
def start(label, request, pick_min, pick_max):
    new_label = np.copy(label)
    if (N in request):
        distance = 0  # 0 time dist. from and to depot
    else:
        distance = calc_dist(lats[request[0]], lats[request[1]],
                             lons[request[0]], lons[request[1]])
        '''
            possible: real distance with routing API: (routing fct defined further down)
            distance = routing(lats[request[0]],lons[request[0]],lats[request[1]],lons[request[1]])
        
        '''
        new_label[7] = 0  # set sleep label to 0
    loaded = 1
    if (request[2] == None):  # not an existing request (empty drive inbetween)
        loaded = 0

    new_label[0] += 0.8 * distance + 0.3 * distance * loaded  # waiting and empty drives are more expensive than driving
    new_label[2] = loaded    # empty or loaded drive
    new_label[3] = distance  # set dist. in minutes to next location
    new_label[8] = 0         # waiting time reset
    new_label[9] = max(0, (label[1] - pick_min) - service)  # calc. possible backward shift
    new_label[10] = pick_max - (label[1] - service)         # calc. possible forward shift
    return (new_label)

def drive(label):
    new_label = np.copy(label)
    Delta = min(label[3], 540 - label[4], 780 - label[5], 270 - label[6])
    new_label[1] = label[1] + Delta  # time
    new_label[3] = label[3] - Delta  # dist
    new_label[4] = label[4] + Delta  # rest
    new_label[5] = label[5] + Delta  # shift
    new_label[6] = label[6] + Delta  # break
    return (new_label)

def break_(label, Delta):
    new_label = np.copy(label)
    new_label[0] += Delta
    new_label[1] = label[1] + Delta  # time
    new_label[5] = label[5] + Delta  # shift
    new_label[6] = 0  # break
    return (new_label)

def rest(label, Delta):
    new_label = np.copy(label)
    new_label[0] += Delta
    new_label[1] = label[1] + Delta  # time
    new_label[4] = 0  # rest
    new_label[5] = 0  # shift
    new_label[6] = 0  # break
    new_label[7] = 1  # sleep
    return (new_label)

def wait(label, delivery_buffer, t_min, t_max, prev_sleep, non_depot,
         prev_forward, prev_backward):
    c = 1  # feasibility crit: 1 = feasible
    prev_buffer = prev_forward + prev_backward
    new_label = np.copy(label)
    wait_time = max(0, t_min - label[1]) * non_depot
    shift_time = max(780 - label[5] - 90, 0)
    forward_shift = min(label[10], max(delivery_buffer, 0))
    not_addable_sleep = max(0, wait_time - forward_shift)
    if wait_time > 0:
        if (label[7] == 0):  # if we already slept, wait will be ignored
            shift_buffer = shift_time - wait_time
            if wait_time < 45:  # wait < 45 not long enough to count as break
                shift_buffer = min(shift_time, 270 - label[6]) - wait_time

            if t_min - label[
                    1] > 660:  # if we have to wait over 660min anyway, we'll sleep
                new_label = rest(label, wait_time)
                new_label[9] = min(new_label[9],
                                   max(0, (t_max - new_label[1]) - 660))
                forward_shift = min(forward_shift, t_max - label[1])
                wait_time = 0

            elif (not_addable_sleep <= shift_time):
                # we can forward shift the whole tour!
                wait_time = not_addable_sleep
                new_label[0] += wait_time * non_depot
                new_label[1] += wait_time
                new_label[5] += wait_time
                new_label[8] = wait_time
                new_label[9] = 0
                forward_shift = max(0, forward_shift - wait_time)

            elif (prev_forward >= wait_time and prev_sleep == 1):
                # we shift the start of this single drive
                new_label[
                    0] += wait_time * non_depot  # by making previous sleep longer
                wait_time = not_addable_sleep  # possible if we slept before and wait for the rest
                new_label[5] += wait_time
                new_label[8] = not_addable_sleep
                new_label[9] = 0
                forward_shift = max(0, forward_shift - wait_time)

            elif (shift_buffer >=
                  0):  # if enough shift time to wait we just wait
                #   print('we just wait')
                new_label[0] += wait_time * non_depot
                new_label[5] += wait_time
                new_label[9] = min(label[9], shift_buffer)

            elif t_max - label[
                    1] > 660:  # if we have enough time to sleep, we'll sleep
                new_label = rest(label, max(660, wait_time))
                new_label[0] = label[0] * (
                    1 - non_depot) + non_depot * new_label[0]
                new_label[9] = min(new_label[9],
                                   max(0, (t_max - new_label[1]) - 660))
                forward_shift = min(forward_shift, t_max - label[1])
                wait_time = 0

            else:
                c = 0  # else -> not feasible
        else:
            if t_max - label[  # if we have enough time to sleep
                    1] > 660:  # we'll still sleep now!
                new_label = rest(label, max(660, wait_time))
                new_label[0] = label[0] * (
                    1 - non_depot) + non_depot * new_label[0]
                new_label[9] = min(new_label[9],
                                   max(0, (t_max - new_label[1]) - 660))
                forward_shift = min(label[10], t_max - label[1])
                wait_time = 0
            else:
                new_label[
                    0] += wait_time * non_depot  # wait time is added to last sleep time (costs still apply)

    new_label[10] = forward_shift
    new_label[8] = wait_time
    return (new_label, c)

def visit(label, request, t_min, t_max, pick_max, prev_sleep, prev_forward,
          prev_backward):
    c = 1
    non_depot = 1
    if N in request:  # 0 dist from and to depot
        non_depot = 0


## Feasibility Tests:
    delivery_buffer = t_max - label[1]
    if (delivery_buffer < 0):  # we can't reach the service window
        c = 0
    if c == 1:
        new_label, c = wait(label, delivery_buffer, t_min, t_max, prev_sleep,
                            non_depot, prev_forward,
                            prev_backward)  # calling wait function
        if c == 1:
            if (new_label[5] + 90 > t_max):  # not enough time for service
                c = 0
    else:
        new_label = np.copy(label)
    new_label[0] = new_label[0] * c - (1 - c)  # neg costs if not feasible
    new_label[1] = max(new_label[1], t_min) + service
    new_label[5] += service  # service time counts towards shift
    new_label[6] = 0  # service time resets break
    return (new_label)


### Labeling Algorithm
Function combine combines an existing route with a request at the end, or returns not feasible. Using label_updates that loops over for necessary actions (REF's like driving and resting until the next destination is reached.

In [55]:
def combine(route, request, arrival_only=0):
    '''
    TAKES A ROUTE AND ADDS A REQUEST AT THE END
    Starting with a drive from last point to the startpoint of the new route 
    and then adding the route, all by calling label_updates()
    Returns the initial label and a 0 if not a possible combination
    Ex.: combine([(40, 40, 20), [ 0, 940,    0,    0,  0,  0,  0,  0,  0 , 10080,  10080]],(2, 4, 1))
    
    If simple=1 : only the drive to the request is calculated and not the request itself
    '''

    route2 = list.copy(route)
    initial_label = route[-1]
    last_r = route[-2]  # this is always a real request
    first_trip = (last_r[1], request[0], None)

    t_min = pick_start[request[2]]
    t_max = pick_end[request[2]]
    pickup_min = delivery_start[last_r[2]]
    pickup_max = delivery_end[last_r[2]]
    prev_sleep = initial_label[7]
    prev_forward = initial_label[10]
    prev_backward = initial_label[9]

    label, feasible = label_updates(initial_label, first_trip, pickup_min,
                                    pickup_max, t_min, t_max, prev_sleep,
                                    prev_forward, prev_backward)
    if feasible == 0:
        return initial_label, 0
    route2.append(first_trip)
    route2.append(label.astype(int))

    if arrival_only == 1:
        return route2, feasible

    pickup_min = pick_start[request[2]]
    pickup_max = pick_end[request[2]]
    t_min = delivery_start[request[2]]
    t_max = delivery_end[request[2]]
    prev_sleep = label[7]
    prev_forward = label[10]
    prev_backward = label[9]

    end_label, feasible = label_updates(label, request, pickup_min, pickup_max,
                                        t_min, t_max, prev_sleep, prev_forward,
                                        prev_backward)
    if feasible == 0:
        return initial_label, 0

    route2.append(request)
    route2.append(end_label.astype(int))

    return route2, feasible


def label_updates(initial_label, request, pickup_min, pickup_max, t_min, t_max,
                  prev_sleep, prev_forward, prev_backward):
    # returns the new label at the end of the request and "1"
    # or the old label and "0" if adding the request is not possible

    N = len(lats)
    if (N in request):
        dist = 0  # 0 dist from and to fictional depot
    else:
        dist = calc_dist(lats[request[0]], lats[request[1]], lons[request[0]],
                         lons[request[1]])

    # 1st check if its possible via lower bounds:
    k_rest = np.floor(dist / 540)
    k_break = np.floor(dist / 270) - k_rest

    # time window cannot be reached:
    if initial_label[1] + dist + 660 * k_rest + 45 * k_break > t_max:
        return (initial_label, 0)

    # else: start labelling algorithm:
    else:
        crit = 0
        new_label = start(initial_label, request, pickup_min, pickup_max)

        while (crit == 0):
            drive_label = drive(new_label)
            if drive_label[3] <= 0:
                end_label = visit(drive_label, request, t_min, t_max,
                                  pickup_max, prev_sleep, prev_forward,
                                  prev_backward)
                feasible = 1
                if end_label[0] < 0:
                    feasible = 0
                    end_label = np.copy(initial_label)
                return (end_label, feasible)
                crit = 1

            if (drive_label[4] == 540 or drive_label[5] == 780
                    or (540 - drive_label[4] <= drive_label[3] <= 270)):
                new_label = rest(drive_label, 660)
            else:
                new_label = break_(drive_label, 45)

        end_label = visit(new_label, request, t_min, t_max, pickup_max,
                          prev_sleep, prev_forward, prev_backward)
        feasible = 1
        if end_label[0] < 0:
            feasible = 0
            end_label = np.copy(initial_label)

        return (end_label, feasible)

### Creating an Initial Solution

In [38]:
def simple_sol(requs, lats, lons, pick_start, pick_end, delivery_start, delivery_end ):
    '''
    Puts all requests in individual route
    Input: List of all requests in the form ((i,j),(l,m),...)
    Output: initial_routes = 
    ''' 
    n = len(requs)
    N = n*2  # number of locations = depotnumber (loc starting at 0)
    start_node = (N,N,n)    
    start_label = [0,0,0,0,0,0,0,1,0,10080,10080]
    global initial_routes 
    initial_routes = []
    valid_requests = []
    
    for r in requs:
        route, feasibility = combine([start_node,start_label],r)
        if feasibility == 1:
            initial_routes.append(route)
            valid_requests.append(r)
    return(initial_routes, valid_requests)

In [39]:
def greedy_split(initial_routes, fixcosts=2000, mode=1, seconds= 300):
    '''
        splits a dataset and applies the greedy algorithm on the halves 
        (for mode = 1, or thirds for mode = 2) of the request set first
    '''
    
    infeas_after_delete = []
    total_savings3 = 0
    n = len(initial_routes)
    if mode ==1:
        (sol1, function_calls1, total_iter_count1, total_savings1, 
             cost_diff1, changes1, infeas_after_delete1
             ) = simple_greedy(initial_routes[:int(n/2)],fixcosts,0,seconds)

        (sol2, function_calls2, total_iter_count2, total_savings2, 
             cost_diff2, changes2, infeas_after_delete2
             )= simple_greedy(initial_routes[int(n/2):],fixcosts,0,seconds)

        (sol, function_calls, total_iter_count, total_savings, 
             cost_diff, changes, infeas_after_delete
             ) = simple_greedy(sol1+sol2,fixcosts,0,seconds)
        
    else:
        (sol1, function_calls1, total_iter_count1, total_savings1, 
             cost_diff1, changes1, infeas_after_delete1
             ) = simple_greedy(initial_routes[:int(n/3)],fixcosts,0,seconds)
        (sol2, function_calls2, total_iter_count2, total_savings2, 
             cost_diff2, changes2, infeas_after_delete2
             )= simple_greedy(initial_routes[int(n/3):int(2*n/3)],fixcosts,0,seconds)
        (sol3, function_calls3, total_iter_count3, total_savings3, 
             cost_diff3, changes3, infeas_after_delete3
             )= simple_greedy(initial_routes[int(2*n/3):],fixcosts,0,seconds)
        (sol, function_calls, total_iter_count, total_savings, 
             cost_diff, changes, infeas_after_delete
             ) = simple_greedy(list(sol1)+list(sol2)+list(sol3),fixcosts,0, seconds)
        
    savings = total_savings3+total_savings1+total_savings2+total_savings
    
    return sol, infeas_after_delete, savings 

### All in one - reading in data and generating a single request initial sol:

In [40]:
def data_read(Sheetname, visualization = 0, printing = 1, filepath = r'/Data.xlsx'):
    # INITIALIZE REQUEST-DATA

    # visualization = 1 plots all request routes
    # printing = 1 prints initial sol

    crit = 1           # stopping crit for while loop
    sheetname = Sheetname 
    global service 
    service = 90
    global N
    global lats
    global lons 
    global pick_start
    global pick_end 
    global delivery_start
    global delivery_end 
    
    n, requests, lats, lons, pick_start, pick_end, delivery_start, delivery_end = read_data(filepath, sheetname)
    N = 2*n
    coordinates = [[lats],[lons]]
    time_windows = [[[pick_start],[pick_end]],[[delivery_start],[delivery_end]]]

    # GENERATE INITIAL SOLUTION
    initial_routes, valid_requests = simple_sol(requests,lats, lons, pick_start, pick_end, delivery_start, delivery_end )
    l = len(initial_routes)
    if l<n:
        print('Error, not all routes are initially feasible!')
    if printing == 1:
        print_requ(initial_routes)

    # OUTPUT
    if visualization == 1:
        fig, axs = plt.subplots()#figsize=(6,6)
        axs.set_aspect('equal', 'datalim') 
        xs, ys = europe_polygon.exterior.xy 
        k=0
        jet= plt.get_cmap('jet')
        colors = iter(jet(np.linspace(0,1,n)))
        axs.fill(xs, ys, alpha=0.5, ec='none')

        for i in range(n):
            x = lons[2*i]
            lon = lons[2*i+1]
            y = lats[2*i] 
            lat = lats[2*i+1]
            axs.scatter(x, y, color='g',marker = ">")
            axs.scatter(lon, lat, color='r',marker = 'd')
            axs.plot([lon,x], [lat,y],label = 'r.'+str(i) ,color=next(colors))
            axs.legend()
            k+=1
        plt.show()
    return(n, requests, lats, lons, pick_start, pick_end, delivery_start, delivery_end, coordinates,
           time_windows,initial_routes, valid_requests)


## 3. Neighborhood Operators: 
    

In [51]:
def arc_costs(all_routes, i, j, extra_output=0, mode=0, fixcosts=2000):
    ''' 
        Outputs the cost saving estimations of a move
        (D&I, SWAP,..)
        i < j! where i and j are the request position, NOT the request number
        (est. bc we don't count the possible reschedule/cost s(avings) in requests further down)
            
         We only swap if different tours and not both single tours
         We check also if front insertion is possible, if request before is a depot

        mode = 0: use all implemented moves
        mode = 1: using only (reverse) d&i and swap (like in cim paper)
        mode = 2: mode 1 + behind (reverse) d&i
        mode = 3: mode 1 + multiple swap
    '''

    n = len(initial_routes)
    lengths = []  # all tour lengths (= # requests) in solution
    swap_inf = 1
    insert_behind = 1
    reverse_infront = 1

    if mode == 0:
        insert_infront = 1
        reverse_behind = 1
        mult_swap_inf = 1
    elif mode == 1:
        insert_infront = np.inf
        reverse_behind = np.inf
        mult_swap_inf = np.inf
    elif mode == 2:
        insert_infront = 1
        reverse_behind = 1
        mult_swap_inf = np.inf
    elif mode == 3:
        insert_infront = 1
        reverse_behind = 1
        mult_swap_inf = 1
        #mult_d_i = np.inf

    for k in all_routes:
        lengths.append(
            int((len(k) - 2) / 4)
        )  # -2 for depot, /4 as we have 4 entries per request (2 labels, 2 location numbers)
        # should be = n summed up!
    tour_numbers = np.arange(len(all_routes))
    # example: lengths = [1, 2, 2, 1, 2, 1, 1]
    # -> request order-number 5 = first request in 3rd tour (starting count at 0)
    len_cum = np.cumsum(lengths) - 1
    tour_nr = tour_numbers[len_cum >= i][0]

    if tour_nr == 0:
        index = 0
    else:
        index = len_cum[int(tour_nr - 1)] + 1
    position = i - index
    p = 4 * (position + 1)

    if j == n - 1:
        tour_nr2 = tour_numbers[-1]
    else:
        tour_nr2 = tour_numbers[len_cum >= j][0]
    if tour_nr2 == 0:
        index2 = 0
    else:
        index2 = len_cum[int(tour_nr2 - 1)] + 1
    position2 = j - index2
    p2 = 4 * (position2 + 1)

    tour = all_routes[tour_nr]
    tour2 = all_routes[tour_nr2]
    route = np.array(all_routes[tour_nr])
    route1 = np.array(all_routes[tour_nr2])

    route_len = len(route)

    requests1 = route[4::4]
    r = requests1[position]
    requests2 = route1[4::4]
    r2 = requests2[position2]

    if tour_nr == tour_nr2:
        swap_inf = 1  # no swaps in same tour

    # Check if we would delete the tour of i after removal
    route_del = 0
    if len(requests1) == 1:
        route_del = 1
    route_del2 = 0
    if len(requests2
           ) == 1:  #route del for request j if we look at reverse insert
        route_del2 = 1

    # check edge cases:
    depot = tour[0]
    if position == 0:
        pre_r = depot
    else:
        pre_r = requests1[position - 1]

    if position2 == 0:
        pre_r2 = depot
    else:
        pre_r2 = requests2[position2 - 1]
        insert_infront = np.inf  #we don't have to check costs here, 
                                 #they were calculated with i and j = pre_r2 & behind insert

    if position + 1 == lengths[tour_nr]:
        post_r = depot
    else:
        post_r = requests1[position + 1]
        reverse_behind = np.inf

    if position2 + 1 == lengths[tour_nr2]:
        post_r2 = depot
    else:
        post_r2 = requests2[position2 + 1]

    # we ignore the arcs & costs (of 0) for single tour swaps
    if (pre_r2 == pre_r and post_r == post_r2):
        reverse_behind = np.inf  # for single tours reverse results in the same values
        reverse_infront = np.inf
        swap_inf = 1

    ## time windows overlapping at all?
    pick_i = [pick_start[r[2]], pick_end[r[2]]]
    del_i = [delivery_start[r[2]], delivery_end[r[2]]
             ]  # are they overlapping with the window j leaves free?
    pick_j = [pick_start[r2[2]], pick_end[r2[2]]]
    del_j = [delivery_start[r2[2]], delivery_end[r2[2]]]
    i_del_window = [delivery_start[pre_r[2]], pick_end[post_r[2]]]
    j_del_window = [delivery_start[pre_r2[2]], pick_end[post_r2[2]]]

    if i_del_window[0] >= pick_j[1] or i_del_window[1] <= del_j[0]:
        swap_inf = 1
    elif j_del_window[0] >= pick_i[1] or j_del_window[1] <= del_i[0]:
        swap_inf = 1
    if del_j[0] >= pick_i[1]:
        insert_behind = np.inf  # i cannot follow j
        reverse_infront = np.inf

    elif del_i[0] >= pick_j[1]:
        insert_infront = np.inf
        reverse_behind = np.inf

    if position + 1 >= lengths[tour_nr]:
        c_ik = tour[p + 1][0]  # depot behind, so old costs = costs after i
    else:
        c_ik = tour[p + 3][0]

    if position2 + 1 >= lengths[tour_nr2]:
        c_jn = tour2[p2 + 1][0]  # depot behind, so old costs = costs after j
    else:
        c_jn = tour2[p2 + 3][0]

    if swap_inf == 0:
        route_lj, f1 = combine(list(tour[:p - 2]), r2)  #add j to route:i-1
        if f1 == 0:
            swap_costs = np.inf
        else:
            route_jk, f2 = combine(list(route_lj), post_r, arrival_only=1)
            if f2 == 0:
                swap_costs = np.inf
            else:
                c_ljk = route_jk[-1][0]
                route_mi, f3 = combine(list(tour2[:p2 - 2]), r)
                if f3 == 0:
                    swap_costs = np.inf
                else:
                    route_in, f4 = combine(list(route_mi),
                                           post_r2,
                                           arrival_only=1)
                    if f4 == 0:
                        swap_costs = np.inf
                    else:
                        c_min = route_in[-1][0]
                        ## cost changes from possible shifts after the insertion are disregarded for this estimation
                        swap_costs = (c_ljk - c_ik) + (
                            c_min - c_jn
                        )  # expected change (estimation) of the objective fct / overall costs

        # now multiple swap is also possible, if there are requests behind
        requests_behind = route[p:][4::4]
        requests_behind2 = route1[p2:][4::4]
        number_of_r = len(requests_behind)
        number_of_r2 = len(requests_behind2)

        if mult_swap_inf == 1 and (number_of_r > 0 or number_of_r2 > 0):
            # try 2er swap:
            # first we'll delete i and j,j+1 and remember the costs
            if number_of_r == 0:  # = i has no requ. behind
                j_1 = requests_behind2[0]  # request that will be swapped too
                if number_of_r2 > 1:
                    j_2 = requests_behind2[1]  # request behind swapping
                    costs_j_2_before = tour2[p2 + 7][0]
                else:
                    j_2 = depot
                    costs_j_2_before = tour2[p2 + 3][0]
            # del_costs_i calc.ed already
                route_mi, f1 = combine(list(tour2[:p2 - 2]),
                                       r)  # route without j and j+1 but plus i
                if f1 == 0:
                    mult_swap_costs = np.inf
                else:
                    route_ij2, f2 = combine(
                        route_mi, j_2,
                        arrival_only=1)  # now adding also j+2 back
                    if f2 == 0:
                        mult_swap_costs = np.inf
                    else:
                        route_j_costs = (route_ij2[-1][0] - costs_j_2_before)
                        # now for route i:
                        route_lj, f3 = combine(list(tour[:p - 2]),
                                               r2)  # add j to route i
                        if f3 == 0:
                            mult_swap_costs = np.inf
                        else:
                            route_ljj1, f4 = combine(route_lj, j_1)  # add j+1
                            if f4 == 0:
                                mult_swap_costs = np.inf
                            else:
                                route_j1k, f5 = combine(
                                    route_ljj1, j_2, arrival_only=1)  # add j+1
                                if f5 == 0:
                                    mult_swap_costs = np.inf
                                else:
                                    route_i_costs = (route_j1k[-1][0] - c_ik)
                                    mult_swap_costs = route_i_costs + route_j_costs

            # in this case we'll delete i,i+1 and j and remember the costs
            elif number_of_r2 == 0:
                i_1 = requests_behind[0]  # request that will be swapped too
                if number_of_r > 1:
                    i_2 = requests_behind[1]  # request behind swapping
                    costs_i_2_before = tour[p + 7][0]
                else:
                    i_2 = depot
                    costs_i_2_before = tour[p + 3][0]
                route_lj, f1 = combine(list(
                    tour[:p - 2]), r2)  # route without i and i+1 but plus j
                if f1 == 0:
                    mult_swap_costs = np.inf
                else:
                    route_ji2, f2 = combine(
                        route_lj, i_2,
                        arrival_only=1)  # now adding also i+2 back
                    if f2 == 0:
                        mult_swap_costs = np.inf
                    else:
                        route_i_costs = (route_ji2[-1][0] - costs_i_2_before)
                        # now for route j:
                        route_mi, f3 = combine(list(tour2[:p2 - 2]),
                                               r)  # add j to route i
                        if f3 == 0:
                            mult_swap_costs = np.inf
                        else:
                            route_mii1, f4 = combine(route_mi, i_1)  # add j+1
                            if f4 == 0:
                                mult_swap_costs = np.inf
                            else:
                                route_i1i2, f5 = combine(
                                    route_mii1, i_2, arrival_only=1)  # add j+1
                                if f5 == 0:
                                    mult_swap_costs = np.inf
                                else:
                                    route_j_costs = (route_i1i2[-1][0] - c_jn)
                                    mult_swap_costs = route_i_costs + route_j_costs

            # in this case we'll delete i,i+1 and j,j+1 and remember the costs
            else:
                i_1 = requests_behind[0]  # requests that will be swapped too
                if number_of_r > 1:
                    i_2 = requests_behind[1]  # request behind swapping
                    costs_i_2_before = tour[p + 7][0]
                else:
                    i_2 = depot
                    costs_i_2_before = tour[p + 3][0]
                j_1 = requests_behind2[0]  # requests that will be swapped too
                if number_of_r2 > 1:
                    j_2 = requests_behind2[1]  # request behind swapping
                    costs_j_2_before = tour2[p2 + 7][0]
                else:
                    j_2 = depot
                    costs_j_2_before = tour2[p2 + 3][0]

                route_lj, f1 = combine(list(
                    tour[:p - 2]), r2)  # route without i and i+1 but plus j
                if f1 == 0:
                    mult_swap_costs = np.inf
                else:
                    route_jj1, f2 = combine(route_lj, j_1)  # adding also j+1
                    if f2 == 0:
                        mult_swap_costs = np.inf
                    else:
                        route_j1i2, f3 = combine(
                            route_jj1, i_2,
                            arrival_only=1)  # now adding i+2 back
                        if f3 == 0:
                            mult_swap_costs = np.inf
                        else:
                            route_i_costs = (route_j1i2[-1][0] -
                                             costs_i_2_before)
                            # now for route j:
                            route_mi, f4 = combine(list(tour2[:p2 - 2]),
                                                   r)  # add j to route i
                            if f4 == 0:
                                mult_swap_costs = np.inf
                            else:
                                route_mii1, f5 = combine(route_mi,
                                                         j_1)  # add j+1
                                if f5 == 0:
                                    mult_swap_costs = np.inf
                                else:
                                    route_i1i2, f6 = combine(
                                        route_mii1, i_2, arrival_only=1
                                    )  # add i+2 to check costs
                                    if f6 == 0:
                                        mult_swap_costs = np.inf
                                    else:
                                        route_j_costs = (route_i1i2[-1][0] -
                                                         costs_j_2_before)
                                        mult_swap_costs = route_i_costs + route_j_costs
        else:
            mult_swap_costs = np.inf
    else:
        swap_costs = np.inf
        mult_swap_costs = np.inf

    ## calc delete and insert costs: (also for 2er-d&i)
    if insert_behind == 1:
        route_lk, f = combine(list(tour[:p - 2]), post_r,
                              arrival_only=1)  # (without i)
        if f == 0:
            insertion_costs = np.inf
        else:
            deletion_costs_lk = route_lk[-1][0] - c_ik
            route_ji, f1 = combine(list(tour2[:p2 + 2]), r)  # adding i after j
            if f1 == 0:
                insertion_costs = np.inf
            else:
                if post_r2 == depot:
                    cost_change = route_ji[-1][0] - tour2[p2 + 1][0]
                    insertion_costs = cost_change + deletion_costs_lk - fixcosts * route_del
                else:
                    # compare costs[j+1] vs costs[i+1] after adding i (m-j-i-n)
                    route_in, f2 = combine(
                        route_ji, post_r2,
                        arrival_only=1)  # adding n(arrival) after i
                    if f2 == 0:
                        insertion_costs = np.inf
                    else:
                        cost_change = route_in[-1][
                            0] - c_jn  # comparing costs at n before and after (+3 = arrival label n)
                        insertion_costs = cost_change + deletion_costs_lk - fixcosts * route_del  #cost_change - del_costs_i + driving_costs_lk

    elif insert_infront == 1:
        route_lk, f = combine(list(tour[:p - 2]), post_r,
                              arrival_only=1)  # deletion of i
        if f == 0:
            insertion_costs = np.inf
        else:
            driving_costs_lk = route_lk[-1][0] - c_ik
            route_mi, f1 = combine(list(tour2[:p2 - 2]),
                                   r)  # adding i **before** j (m-i-j-n)
            if f1 == 0:
                insertion_costs = np.inf
            else:  #adding j:
                route_mij, f2 = combine(route_mi, r2, arrival_only=1)
                if f2 == 0:
                    insertion_costs = np.inf
                else:
                    cost_change = route_mij[-1][0] - tour2[p2 - 1][
                        0]  # cost difference taken at j arrival
                    insertion_costs = cost_change + driving_costs_lk - fixcosts * route_del

    else:
        insertion_costs = np.inf

    # reverse insterion::
    if reverse_behind == 1:
        route_mn, f = combine(list(tour2[:p2 - 2]), post_r2,
                              arrival_only=1)  # (without j)
        if f == 0:
            reverse_costs = np.inf
        else:
            driving_costs_mn = route_mn[-1][0] - c_jn
            route_ij, f1 = combine(list(tour[:p + 2]), r2)  # adding j after i

            if f1 == 0:
                reverse_costs = np.inf
            else:
                if post_r == depot:
                    cost_change = route_ij[-1][0] - tour[p + 1][0]
                    reverse_costs = cost_change + driving_costs_mn - fixcosts * route_del2
                else:
                    # compare costs[j+1] vs costs[i+1] after adding j (l-i-j-k)
                    route_jk, f2 = combine(
                        route_ij, post_r,
                        arrival_only=1)  # adding n(arrival) after i
                    if f2 == 0:
                        reverse_costs = np.inf
                    else:
                        r_cost_change = route_jk[-1][
                            0] - c_ik  # comparing costs at n before and after (+3 = arrival label n)
                        reverse_costs = r_cost_change + driving_costs_mn - fixcosts * route_del2

    ## new: j deleted and inserted infront of i
    elif reverse_infront == 1:
        route_mn, f = combine(list(tour2[:p2 - 2]), post_r2,
                              arrival_only=1)  # deletion of i
        if f == 0:
            reverse_costs = np.inf
        else:
            driving_costs_mn = route_mn[-1][0] - c_jn
            route_lj, f1 = combine(list(tour[:p - 2]),
                                   r2)  # adding j **before** i (l-j-i)
            if f1 == 0:
                reverse_costs = np.inf
            else:  #adding i:
                route_lji, f2 = combine(route_lj, r, arrival_only=1)
                if f2 == 0:
                    reverse_costs = np.inf
                else:
                    r_cost_change = route_lji[-1][0] - tour[p - 1][
                        0]  # cost difference taken at j arrival
                    reverse_costs = r_cost_change + driving_costs_mn - fixcosts * route_del2

    else:
        reverse_costs = np.inf

    c_list = [
        swap_costs, mult_swap_costs, insertion_costs * insert_infront,
        insertion_costs * insert_behind, reverse_costs * reverse_infront,
        reverse_costs * reverse_behind
    ]
    c_list = [0 if x != x else x for x in c_list]
    cost_vector = np.array(c_list)
    moves = np.array([
        f'swap({i},{j},parent_sol)', f'swap({i},{j},parent_sol,multi=1)',
        f'insertion({i},{j},parent_sol,front = 1)',
        f'insertion({i},{j},parent_sol)',
        f'insertion({j},{i},parent_sol, front = 1)',
        f'insertion({j},{i},parent_sol)'
    ])
    cost_vector[cost_vector == -np.inf] = np.inf
    first_best_option = np.argmin(cost_vector)
    move1 = moves[first_best_option]
    costs1 = min(cost_vector)
    ss = sorted(set(cost_vector))
    if len(ss) > 1:
        costs2 = ss[1]
        move2 = moves[c_list.index(costs2)]
    else:
        costs2 = np.inf
        move2 = None

    return costs1, move1, costs2, move2  #cost_change_total, move

In [53]:
def delete_and_insert(N, routes, fixcosts, extra_output=0):
    '''
    Computes the solution and costs of a Delte & insert for all 
    combinations/positions from all tours. Just the best sol is kept for all tours
    
    A tour is a squence of routes one driver 'dispatches',
    here we are using route when a part of a tour is meant
    A request (a,b,c) is a job given to a driver, to drive from a to b, 
    c is the job number. (c='None' if it is a drive between requests)
    '''
    try:
        infeas_after_delete
    except NameError:
        infeas_after_delete = []

    n = len(routes)  # current number of individual tours
    delete_tour = np.empty(n, dtype=object)
    insert_tour = np.empty(n, dtype=object)
    costs = []
    trip_len = []
    sol_space = []
    array_routes = np.array(routes)
    i_label = [(N, N, int(N / 2)), [0, 0, 0, 0, 0, 0, 0, 1, 0, 10080,
                                    10080]]  # N = number of requests in total
    cost_diff = []
    labels = np.array(n)
    iter_counter = 0
    valid_counter = 0
    f_c = 0

    for i in range(n):  # going through all tours to calc current costs
        costs.append(routes[i][-1][0])  # current tour costs
        f_c += fixcosts
    base_costs = sum(costs) + f_c  # + added fixed costs = total costs

    for tour in range(n):  # going through all tours
        skip = 0
        all_tours = list.copy(routes)
        best_savings = 0
        indices = [*range(n)]
        indices.remove(tour)

        route = np.array(routes[tour])
        route_len = len(route)

        route_del = max(7 - route_len, 0)
        request_places = np.array(
            range(int(route_len / 4) +
                  1)) * 4  # every 4th entry is an ex. request
        requests1 = route[4::4]

        for r in range(len(requests1)):  # going through all requests of this tour (except depot)
            current_requ = requests1[
                r]  # request that is deleted & inserted elsewhere
            sol_route = current_requ
            if r < len(requests1):
                front_route = list.copy(route[:(request_places[r] +
                                                2)].tolist())  # tour split
            else:
                front_route = list.copy(route.tolist())
            add_r = requests1[
                r + 1:]  # all labels that have to be added afterwards
            new_tour = list.copy(front_route)
            costs_new_tour = new_tour[-1][0]
            if len(add_r) > 0:
                for i in add_r:  # adding all other requests except r back:
                    new_tour, f = combine(front_route, i)
                    front_route = np.copy(new_tour).tolist()
                    if f == 0:
                        break
                if f == 1:
                    costs_new_tour = new_tour[-1][0]
                else:
                    # not feasible anymore
                    costs_new_tour = np.inf
                    skip = 1
                    infeas_after_delete.append(current_requ)
                    # infeas. can happen while delting and building new tour

            if skip == 0:
                for j in indices:  # leaving the current tour out from where the request
                    # was deleted, adding the current request to all other tours
                    current_route = np.array(routes[j])
                    route_len = len(current_route)
                    insert_places = np.array(range(int(route_len / 4) + 1)) * 4
                    labels = current_route[insert_places[1:]]

                    for k in insert_places:  # going through all insert places in tour j
                        iter_counter += 1
                        front_route_insert = current_route[:k + 2].tolist()
                        add_requests = labels[int(k / 4):]
                        new_r, f = combine(front_route_insert, current_requ)

                        if f == 1:  # if adding the del. requ. was possible, try adding the rest of the requests:
                            new_route = new_r
                            for ar in add_requests:
                                iter_counter += 1
                                route_tmp, f = combine(new_route, ar)
                                if f == 1:
                                    new_route = list.copy(route_tmp)
                                else:
                                    break
                            if f == 1:
                                label_tmp = new_route[-1]
                                trip_costs = label_tmp[0]
                                current_costs = (base_costs - costs[tour] +
                                                 costs_new_tour + trip_costs -
                                                 costs[j] - route_del * fixcosts)
                                total_savings = base_costs - current_costs

                                if total_savings > best_savings:
                                    all_tours = list.copy(routes)
                                    best_savings = np.copy(total_savings)
                                    if extra_output == 1:
                                        delete_tour[tour] = [all_tours[tour]]  #tour before delete
                                        insert_tour[tour] = [new_route]
                                    if route_del == 0:
                                        all_tours[tour] = np.copy(new_tour)
                                        all_tours[j] = np.copy(new_route)
                                    else:
                                        all_tours = array_routes[
                                            indices].tolist()
                                        # deleting the tour from this sol. if its empty
                                        if j < tour:
                                            all_tours[j] = list.copy(new_route)
                                        else:
                                            all_tours[j -1] = list.copy(new_route)
                                    sol_route = np.copy(new_route)
                                valid_counter += 1

        # keep only the best result for each request:
        sol_space.append(all_tours)
        cost_diff.append(int(best_savings))
    if extra_output == 1:
        return (sol_space, cost_diff, iter_counter, valid_counter, delete_tour,
                insert_tour, infeas_after_delete)
    else:
        return (sol_space, cost_diff, iter_counter, valid_counter,
                infeas_after_delete)

In [43]:
def insertion(i, j, routes, front=0, fixcosts=2000):
    '''
    Computes the real costs of deleting i an inserting it behind j
    i <  are request numbers that are tour ordered != original request nr!
    '''

    n = len(initial_routes)
    m = len(routes)  # current number of individual tours
    route_del = 0
    costs = []
    f_c = 0
    for q in range(m):  # going through all tours to calc current costs
        costs.append(routes[q][-1][0])  # current tour costs
        f_c += fixcosts
    base_costs = sum(costs) + f_c
    routes_list = list(routes)
    # first: in which tours are i, j?
    lengths = []
    for k in routes:
        lengths.append(int((len(k) - 2) / 4))

    tour_numbers = np.arange(len(routes))
    len_cum = np.cumsum(lengths) - 1
    tour_nr = tour_numbers[len_cum >= i][0]
    feasible = 0

    if tour_nr == 0:
        index = 0
    else:
        index = len_cum[int(tour_nr - 1)] + 1
    position = i - index  # position in requests

    p = 4 * (position + 1)  # position in label

    if j == n - 1:
        tour_nr2 = tour_numbers[-1]
    else:
        tour_nr2 = tour_numbers[len_cum >= j][0]

    if tour_nr2 == 0:
        index2 = 0
    else:
        index2 = len_cum[int(tour_nr2 - 1)] + 1
    position2 = j - index2
    p2 = 4 * (position2 + 1)
    tour = routes[tour_nr]
    tour2 = routes[tour_nr2]
    route = np.array(tour)
    route1 = np.array(tour2)

    route_len = len(route)
    requests = route[4::4]
    r = requests[position]
    requests2 = route1[4::4]
    r2 = requests2[position2]

    # fist insert j after i -> more fail potential
    # then delete i from its route

    depot = tour[0]
    # requests before and after i:
    if position == 0:
        pre_r = depot
    else:
        pre_r = requests[position - 1]

    if position + 1 == lengths[tour_nr]:
        post_r = depot
    else:
        post_r = requests[position + 1]

    # request after j:
    if position2 + 1 == lengths[tour_nr2]:
        post_r2 = depot
    else:
        post_r2 = requests2[position2 + 1]

    # route after deletion:
    if post_r == depot:
        new_route = tour[:p - 2]
        if pre_r == depot:
            route_del = 1
    else:
        front_route, f1 = combine(tour[:p - 2], post_r)
        if f1 == 0:
            return routes, 0, np.inf
        else:
            ## build whole route without i
            for requ in requests[position + 1:]:
                new_route_t, f = combine(front_route, requ)
                if f == 0:
                    return routes, 0, np.inf
                front_route = np.copy(new_route_t)
            new_route = new_route_t

    if front == 0:
        # insert i after j:
        front_route, f1 = combine(list(tour2[:p2 + 2]), r)  # route_ji
        real_cost_change = np.inf
        if f1 == 0:
            return routes, 0, np.inf
        else:
            if post_r2 == depot:  # we don't have to add requests:
                new_route2 = front_route
                real_cost_change = new_route2[-1][0] + new_route[-1][0] - tour[
                    -1][0] - tour2[-1][0] - route_del * fixcosts
            else:
                ## build whole route with i
                for requ in requests2[
                        position2 + 1:]:  #requ that have to be added behind i
                    new_route2, f = combine(front_route, requ)
                    if f == 0:
                        return routes, 0, np.inf
                    front_route = new_route2
                    cost_change = new_route2[-1][0] + new_route[-1][0] - tour[
                        -1][0] - tour2[-1][0]
                    real_cost_change = cost_change - route_del * fixcosts

    else:  # adding i **before** j:
        route_mi, f1 = combine(list(tour2[:p2 - 2]), r)  # route_mi
        real_cost_change = np.inf
        if f1 == 0:
            return routes, 0, np.inf
        else:
            front_route, f2 = combine(route_mi, r2)  #route_mij
            if f2 == 0:
                routes, 0, np.inf
            else:
                if post_r2 == depot:  # we don't have to add any more requests:
                    new_route2 = front_route
                    real_cost_change = new_route2[-1][0] + new_route[-1][
                        0] - tour[-1][0] - tour2[-1][0] - route_del * fixcosts
                else:
                    ## build whole route with i before j
                    for requ in requests2[
                            position2 +
                            1:]:  #requ that have to be added behind i
                        new_route2, f = combine(front_route, requ)
                        if f == 0:
                            return routes, 0, np.inf
                        front_route = new_route2
                        cost_change = new_route2[-1][0] + new_route[-1][
                            0] - tour[-1][0] - tour2[-1][0]
                        real_cost_change = cost_change - route_del * fixcosts

    new_routes = list.copy(routes_list)

    if real_cost_change < np.inf:
        new_routes[tour_nr2] = list(new_route2)  # changed original tour j
        feasible = 1
        if route_del == 0:
            new_routes[tour_nr] = new_route  # changed original tour i
        else:
            indices = np.arange(m)
            indices = np.delete(indices, tour_nr)
            new_routes = list(np.array(new_routes)
                              [indices])  # deletes tour tour_nr belonging to i

    return new_routes, 1, real_cost_change

In [44]:
def simple_greedy(routes, fixcosts=2000, seconds = 300, max_fct_calls = 5000):
    '''
        Only uses 1 d&i iteratively
        
    ''' 
    timeout_start = time.time()
    N = 2*len(initial_routes)
    total_savings = 0
    sol_space, cost_diff, total_iter_count, valid_counter, infeas_after_delete = delete_and_insert(N,routes,fixcosts)
    max_savings = max(cost_diff)
    total_savings += max_savings
    x = np.array(cost_diff)        # execute the best operation first:
    q = np.argwhere(x == max_savings)
    arg_max = random.choice(q)[0]
    function_calls = 0
    sol = sol_space[arg_max]
    changes = []

    while (max_savings > 0)and(function_calls < max_fct_calls):
                
        if (time.time() > timeout_start + seconds):
            break
            
        sol_space, cost_diff, iter_counter, valid_counter, infeas_after_delete = delete_and_insert(N,sol,fixcosts)#,plotting)#, delete_tour, insert_tour 
        function_calls += 1   
        total_iter_count += iter_counter

        x = np.array(cost_diff)        # execute the best operation first:
        q = np.argwhere(x == (max(cost_diff)))
        arg_max = random.choice(q)[0]   # if multiple enties have same max value we choose randomly
        sol = sol_space[arg_max]
        
        old_savings = max_savings
        max_savings = max(cost_diff)
        total_savings += max_savings 
            
    return(sol, function_calls, total_iter_count, total_savings, cost_diff, changes,infeas_after_delete)

### Improvement Graph Set Up

In [45]:
def generate_graph(initial_sol,n, mode = 0):
    '''
        Question: which structure is the best? list of lists? array? dictionnairies (arc weight?)
        -> I'm trying out networkx graph structure
        
        Creating the improvement graph and calculating the edge costs:
    '''
    routes = initial_sol
    m = len(routes) 
    node_set = []
    depotless_nodes = []
    
    subtour_dict = {}
    subtour = 0
    index = 0
    for tour in routes:
        node_set.append(f'd{subtour}')
        for request in tour[4::4]:
            node_set.append(int(request[2]))
            depotless_nodes.append(int(request[2]))
            subtour_dict[request[2]] = subtour
        subtour += 1
    
    g = nx.DiGraph()
    g.add_node('s')
    g.add_nodes_from(node_set)
    g.add_node('t')
    
    nodeset_st = ['s']+ node_set + ['t']
    
    # 0-edges:
    for i in range(1,len(nodeset_st)-1):
        g.add_edge(nodeset_st[i],nodeset_st[i+1], weight = 0, move = None)
        g.add_edge('s',nodeset_st[i], weight = 0, move = None)
        g.add_edge(nodeset_st[i], 't', weight = 0, move = None )
            

    for k in range(n):   
        for l in range(k+1,n):
            c, m, c2, m2 = arc_costs(routes,k,l,0,mode) # 1st and 2nd best cost est.
                
            node = depotless_nodes[k]
            node_l = depotless_nodes[l]
            
            pos1 = 2 + k + subtour_dict[node]
            pos2 = 2 + l + subtour_dict[node_l]
        
            if c != 0 and c != np.inf and c != -np.inf : 
                if  c2 != np.inf and c2 != -np.inf:
                    weigth_vec = [int(c),int(c2)]
                    move_list = [m, m2]
                else:
                    weigth_vec = [int(c)]
                    move_list = [m]
                if 'reverse' in m:
                    if 'front' in m:
                        g.add_edge(nodeset_st[pos1-1], nodeset_st[pos2+1], weight = weigth_vec, move = move_list)
                    else:
                        g.add_edge(nodeset_st[pos1], nodeset_st[pos2+1], weight = weigth_vec, move = move_list)
                elif 'multi' in m:
                    if 'd' in str(nodeset_st[pos2+1]) or (pos2+2) >= len(nodeset_st):
                        g.add_edge(nodeset_st[pos1-1], nodeset_st[pos2+1], weight = weigth_vec, move = move_list)
                    else:
                        g.add_edge(nodeset_st[pos1-1], nodeset_st[pos2+2], weight = weigth_vec, move = move_list)
                else:
                    if 'front' in m:
                        g.add_edge(nodeset_st[pos1-1], nodeset_st[pos2], weight = weigth_vec, move = move_list)
                    else: g.add_edge(nodeset_st[pos1-1], nodeset_st[pos2+1], weight = weigth_vec, move = move_list)
                        
    return g

In [46]:
def swap(i, j, parent_sol, multi = 0):
    ''' SWAP:
        i < j! where i and j are the request position, NOT the request number
        
         removal of i from l-i-k  &  j from m-j-n
         insertion of j in l-j-k  &  i  in  m-i-n
         
         Ex.:   parent_sol = sol_20
                print_requ(parent_sol)
                sol,f,c =swap(4,6)
                print(f,c)
                print_requ(sol)
    '''       
    n = len(initial_routes)
    lengths = []                          # all tour lengths (= # requests) in solution
    all_routes = np.copy(parent_sol)      # solution from improvement graph node parent
    f = 1
    
    for k in all_routes:
        lengths.append(int((len(k)-2)/4)) 
        
    tour_numbers = np.arange(len(all_routes))

    tour_numbers = np.arange(len(all_routes))
    len_cum = np.cumsum(lengths)-1
    
    tour_nr = tour_numbers[len_cum >= i][0]

    if tour_nr == 0:
        index = 0
    else:
        index = len_cum[int(tour_nr-1)]+1
    position = i - index
    p = 4*(position+1)

    if j == n-1:
        tour_nr2 = tour_numbers[-1]
    else:
        tour_nr2 = tour_numbers[len_cum >= j][0]
  
    if tour_nr2 == 0:
        index2 = 0
    else:
        index2 = len_cum[int(tour_nr2-1)]+1
    position2 = j - index2
    p2 = 4*(position2+1)

    tour = all_routes[tour_nr]
    tour2 = all_routes[tour_nr2]
    route = np.array(all_routes[tour_nr])
    route1 = np.array(all_routes[tour_nr2])

    route_len = len(route)

    requests1 = route[4::4]
    r = requests1[position]
    
    requests2 = route1[4::4]
    r2 = requests2[position2]

    # check edge cases:
    depot = tour[0]
    if position == 0:
        pre_r = depot
    else: pre_r = requests1[position-1]

    if position2 == 0:
        pre_r2 = depot
    else: pre_r2 = requests2[position2-1]

    if position+1 == lengths[tour_nr]:
        post_r = depot
    else:
        post_r = requests1[position+1]
        
    if position2+1 == lengths[tour_nr2]:
        post_r2 = depot
    else:
        post_r2 = requests2[position2+1]

    # we ignore single tour swaps 
    if (pre_r2 == pre_r and post_r == post_r2):
        return all_routes, 0, np.inf
    
    if multi == 0:    
        ## time windows overlapping at all?
        pick_i = [pick_start[r[2]],pick_end[r[2]]]
        del_i  = [delivery_start[r[2]],delivery_end[r[2]]]   # are they overlapping with the window j leaves free?
        pick_j = [pick_start[r2[2]],pick_end[r2[2]]]
        del_j  = [delivery_start[r2[2]],delivery_end[r2[2]]]
        i_del_window = [delivery_start[pre_r[2]],pick_end[post_r[2]]]
        j_del_window = [delivery_start[pre_r2[2]],pick_end[post_r2[2]]]
    
        if i_del_window[0] >=  pick_j[1] or i_del_window[1] <=  del_j[0]:
            return all_routes, 0, np.inf
        if j_del_window[0] >=  pick_i[1] or j_del_window[1] <=  del_i[0]:
            return all_routes, 0, np.inf
            
        ## new routes:
        # for inserting j:
        new_route, f1 = combine(list.copy(list(tour[:p-2])),r2)
        if f1 == 0:
            return all_routes, f1, np.inf

        for request in requests1[position+1:]:
            new_route, f1 = combine(new_route, request)
            if f1 == 0:
                return all_routes, f1, np.inf

        # building tour after inserting i:    
        new_route2, f2 = combine(list.copy(list(tour2[:p2-2])),r)  
        if f2 == 0:
            return all_routes, f2, np.inf

        for request in requests2[position2+1:]:
            new_route2, f = combine(new_route2, request)
            if f == 0:
                return all_routes, f, np.inf

        # swap the changed tours in the parent_sol
        cost_before = all_routes[tour_nr][-1][0]
        cost_before2 = all_routes[tour_nr2][-1][0]

        all_routes[tour_nr]  = new_route   # changed original tour i
        all_routes[tour_nr2] = new_route2  # changed original tour j

        cost_after  = new_route[-1][0]
        cost_after2 = new_route2[-1][0]

        real_cost_change = cost_after + cost_after2 - cost_before - cost_before2
        
        
    else:  # multiple swap! 
        # now multiple swap is also possible, if there are requests behind
        requests_behind = route[p:][4::4]
        requests_behind2 = route1[p2:][4::4]
        number_of_r = len(requests_behind)
        number_of_r2 = len(requests_behind2)
        
        # try 2er swap:
        # first we'll delete i and j,j+1 and remember the costs
        if number_of_r > 1:
            i_2 = requests_behind[1]
        else:
            i_2 = depot
        if number_of_r2 > 1: 
            j_2 = requests_behind2[1]
        else:
            j_2 = depot
            
        # time window overlap?    
        pick_i = [pick_start[r[2]],pick_end[r[2]]]
        del_i2  = [delivery_start[post_r[2]],delivery_end[post_r[2]]]   # are they overlapping with the window j leaves free?
        pick_j = [pick_start[r2[2]],pick_end[r2[2]]]
        del_j2  = [delivery_start[post_r2[2]],delivery_end[post_r2[2]]]
        
        i_delete_window = [delivery_start[pre_r[2]],pick_end[i_2[2]]]
        j_delete_window = [delivery_start[pre_r2[2]],pick_end[j_2[2]]]
        
        if i_delete_window[0] >=  pick_j[1] or i_delete_window[1] <=  del_j2[0]:
            return all_routes, 0, np.inf
        if j_delete_window[0] >=  pick_i[1] or j_delete_window[1] <=  del_i2[0]:
            return all_routes, 0, np.inf                    
        
        front_route_j, f1 = combine(list(tour2[:p2-2]),r)     # route without j and j+1 but plus i
        if f1 == 0:
            return all_routes, 0, np.inf
        
        new_route_j = front_route_j
        for requ in requests_behind2:
            new_route_j, f = combine(front_route_j, requ)
            if f == 0:
                return all_routes, 0, np.inf
            front_route_j = new_route_j
                
        # same for route i:        
        front_route_i, f2 = combine(list(tour[:p-2]),r2)     # route without i and i+1 but plus j
        if f2 == 0:
            return all_routes, 0, np.inf
        
        new_route_i = front_route_i
        for requ in requests_behind:
            new_route_i, f = combine(front_route_i, requ)
            if f == 0:
                return all_routes, 0, np.inf
            front_route_i = new_route_i
            
        # swap the changed tours in the parent_sol
        cost_before = all_routes[tour_nr][-1][0]
        cost_before2 = all_routes[tour_nr2][-1][0]

        all_routes[tour_nr]  = new_route_i   # changed original tour i
        all_routes[tour_nr2] = new_route_j   # changed original tour j

        cost_after  = new_route_i[-1][0]
        cost_after2 = new_route_j[-1][0]

        real_cost_change = cost_after + cost_after2 - cost_before - cost_before2   
                                
    return all_routes, f, real_cost_change

In [47]:
def SPP(g,initial_sol,n):
    
    '''
    Shortest Path Procedure
    
    Takes graph g and an initial solution as input and finds the shortest path 
    wrt. to the edge weights & feasibility of the induced moves 
    
    '''
    m = len(initial_sol)                 # nr of subtours
    
    cost_change_all = 0
    node_set = np.array(g.nodes)
    edge_set = g.edges.data()
    nm = len(node_set)

    node_nr_name = {}
    it = 0
    for i in node_set:
        node_nr_name[i] = it
        it += 1  
    
    solutions = [[] for _ in range(nm)]
    path = [['s']] + [ [] for _ in range(nm-1) ] # list of paths (also a list for each request)
    cost_comparison = []
    
    f_v = np.repeat(0,nm)                        # set function values 0
    parent = np.repeat(0,nm)
    
    for j in range(1,len(node_set)):             # all nodes but s
        i = parent[j]
        node = node_set[j]
        parent_node = node_set[i]
        path[j] = list.copy(path[i])    # paths consist of the order numbered nodes not real request numbers
        path[j].append(node)
        if str.isdecimal(node):
            out_edges_dict = g.out_edges([int(node)], data = True)
        else:
            out_edges_dict = g.out_edges([node], data = True)
        out_edges = []
        for e in out_edges_dict:
            out_edges.append([e[0],e[1],e[2]])
        first = 1
        for k in range(len(out_edges)):
            edge = out_edges[k]            # edge[0:1] gives the nodes, edge[2] the data
            node_k = edge[1]
            edge_costs = edge[2]['weight'] # now possibly 2 values!
            if isinstance(edge_costs,int):
                edge_cost = edge_costs
            else: 
                edge_cost = edge_costs[0]
            f_new = f_v[j] + edge_cost
            pos_k = node_nr_name[str(node_k)]
            
            if f_new < f_v[pos_k]:
                feasible = 1          
                c = 0
                parent_sol = np.copy(solutions[j])

                # add the move from (j,k) to the solution from j (None if no moves)
                adding_moves = edge[2]['move']
                
                if len(parent_sol) == 0:
                    parent_sol = np.copy(initial_sol)
                    
                if adding_moves == None:
                    current_sol = np.copy(parent_sol)
                    f_real = f_new
                else:
                    adding_move = adding_moves[0]
                    # adding_move evals to swap(i,j) or insert(i,j), takes parent_sol as input!
                    current_sol, feasible, c = eval(adding_move)
                    
                    if feasible == 0 and len(adding_moves)==2:
                        if  f_v[j] + edge_costs[1] < f_v[pos_k]: # and we would choose edge still with 2nd best costs:
                            adding_move = adding_moves[1]
                            current_sol, feasible, c = eval(adding_move)
                    
                cost_comparison.append([edge_cost - c]) 
                f_real = f_v[j] + c
                        
                if feasible == 1 and f_real < f_v[pos_k]:  # check if move possible & real costs still better
                    solutions[pos_k] = current_sol
                    f_v[pos_k]= f_real 
                    parent[pos_k] = j
                    cost_change_all += c 

    return solutions[-1], path[-1], f_v[-1], cost_comparison # can be changed, with [-1] we only return best

### Full LNS Algorithm

In [1]:
def VLNS_IG(initial_sol,
           cost_output = 0,
           move_mode = 0,
           fixcosts = 2000,
           max_iter = 100000000):
    '''
    Large Neighborhood Search (implicit) with improvement graph neighborhood navigation
    
    '''
    n = len(initial_routes)
    steps = 0
    g = generate_graph(initial_sol, n, mode=move_mode)

    c_inf = 0
    underest = 0
    overest = 0
    improvement = 0
    max_dev = [0, 0]
    prev_cost_change = 0
    break_crit = 'max iter'

    solution, path, cost_change, cost_comparison = SPP(
        g, initial_sol, n)  
    if cost_change == 0:
        steps = max_iter
        solution = initial_sol

    #  cost_comparison = (estimated  - real costs)  -> neg. =  overestimated savings
    timeout_start = time.time()

    while (steps < max_iter):

        g = generate_graph(solution, n, mode=move_mode)
        new_solution, path, cost_change_new, cost_comp = SPP(g, solution, n)

        steps += 1
        if cost_change_new >= 0:
            break_crit = 'local min'
            break

        improvement = cost_change_new + cost_change
        prev_cost_change = cost_change
        cost_change = cost_change_new

        if cost_output == 1:
            cc = np.array(cost_comp)
            bool_array = np.array(cc == -np.inf)
            inf_entries = cc[bool_array]
            r = np.invert(bool_array)
            try:
                c_inf += len(inf_entries)[0]
            except:
                c_inf += len(inf_entries)
            c_rest = cc[r]
            underest += len(c_rest[c_rest < 0])
            overest += len(c_rest[c_rest > 0])
            max_dev = (max(max_dev[0],
                           max(c_rest)), min(max_dev[1], min(c_rest)))

        if time.time() > timeout_start + 300: #timeout after 5 min
            break
        solution = new_solution
        cost_comparison = [c_inf, underest, overest, max_dev]
        
    new_costs = 0
    for s in solution:
        new_costs += s[-1][0] + fixcosts
        
    return solution, path, cost_change, cost_comparison, improvement, break_crit, new_costs

## 4. Brute Force Solution:

In [29]:
def tuple_table(initial_routes):
    # which tuples are possible combinations
    combi_bool = np.zeros([len(initial_routes),len(initial_routes)])
    i=0
    dreier = 0
    for r in initial_routes:
        j=0
        for r2 in initial_routes:
            route, feas = combine(r,r2[-2])
            if feas==1:
                val = route[-1][0]
                combi_bool[i][j] = feas
            j+=1
        i+=1
    return(combi_bool)

In [4]:
def combs(n,combi_bool,requests):
    zweier = []
    for row in range(n):
        for col in range(n):
            if combi_bool[row][col] == 1:
                sol1, f = combine(initial_routes[row], requests[col])
                if f == 1:
                    zweier.append([row, col])
    dreier = []
    for row in range(n):
        for col in range(n):
            if combi_bool[row][col] == 1:
                for t in range(n):
                    if (combi_bool[row][t] == 1):
                        if (combi_bool[col][t] == 1):
                            sol1, f = combine(initial_routes[row],requests[col])
                            if f == 1:
                                sol2, f = combine(sol1, requests[t])
                                if f == 1:
                                    dreier.append([row, col, t])

    vierer = []
    for d in dreier:
        a, b, c = d
        for t in range(n):
            if (combi_bool[a][t] == 1):
                if (combi_bool[b][t] == 1):
                    if (combi_bool[c][t] == 1):
                        sol1, f = combine(initial_routes[a], requests[b])
                        if f == 1:
                            sol2, f = combine(sol1, requests[c])
                            if f == 1:
                                sol3, f = combine(sol2, requests[t])
                                if f == 1:
                                    vierer.append([a, b, c, t])

    funfer = []
    for v in vierer:
        a, b, c, d = v
        for t in range(n):
            if (combi_bool[a][t] == 1) or (combi_bool[t][a] == 1):
                if (combi_bool[b][t] == 1) or (combi_bool[t][b] == 1):
                    if (combi_bool[c][t] == 1) or (combi_bool[t][c] == 1):
                        if (combi_bool[d][t] == 1) or (combi_bool[t][d] == 1):
                            sol1, f = combine(initial_routes[a], requests[b])
                            if f == 1:
                                sol2, f = combine(sol1, requests[c])
                                if f == 1:
                                    sol3, f = combine(sol2, requests[d])
                                    if f == 1:
                                        sol4, f = combine(sol3, requests[t])
                                        if f == 1:
                                            funfer.append([a, b, c, d, t])

    sechser = []
    for v in funfer:
        a, b, c, d, e = v
        for t in range(n):
            if (combi_bool[a][t] == 1) or (combi_bool[t][a] == 1):
                if (combi_bool[b][t] == 1) or (combi_bool[t][b] == 1):
                    if (combi_bool[c][t] == 1) or (combi_bool[t][c] == 1):
                        if (combi_bool[d][t] == 1) or (combi_bool[t][d] == 1):
                            if (combi_bool[d][t] == 1) or (combi_bool[t][e] == 1):
                                sol1, f = combine(initial_routes[a],requests[b])
                                if f == 1:
                                    sol2, f = combine(sol1, requests[c])
                                    if f == 1:
                                        sol3, f = combine(sol2, requests[d])
                                        if f == 1:
                                            sol4, f = combine(sol3, requests[e])
                                            if f == 1:
                                                sol5, f = combine(sol4, requests[t])
                                                if f == 1:
                                                    sechser.append([a, b, c, d, e, t])

    siebener = []
    for v in sechser:
        a, b, c, d, e, f = v
        for t in range(n):
            if (combi_bool[a][t] == 1) or (combi_bool[t][a] == 1):
                if (combi_bool[b][t] == 1) or (combi_bool[t][b] == 1):
                    if (combi_bool[c][t] == 1) or (combi_bool[t][c] == 1):
                        if (combi_bool[d][t] == 1) or (combi_bool[t][d] == 1):
                            if (combi_bool[d][t] == 1) or (combi_bool[t][e]== 1):
                                if (combi_bool[e][t] == 1) or (combi_bool[t][f]== 1):
                                    sol1, f = combine(initial_routes[a],requests[b])
                                    if f == 1:
                                        sol2, f = combine(sol1, requests[c])
                                        if f == 1:
                                            sol3, f = combine(sol2, requests[d])
                                            if f == 1:
                                                sol4, f = combine(sol3, requests[e])
                                                if f == 1:
                                                    sol5, f = combine(sol4, requests[f])
                                                    if f == 1:
                                                        sol6, f = combine(sol5, requests[t])
                                                        if f == 1:
                                                            siebener.append([a, b, c, d, e,f, t])

    return (zweier, dreier, vierer, funfer, sechser, siebener)

In [6]:
def valid_combs(initial_routes, zweier, dreier, vierer, funfer, requests, fixcosts = 2000):
    
    n = len(initial_routes)
    base_costs = 0
    initial_costs =[]
    for s in initial_routes:
        c = s[-1][0]
        base_costs += c + fixcosts    
        initial_costs.append(c)
       
    valid_vierer = []
    vierer_cost = []
    for v in vierer:
        a,b,c,d = v
        costs = sum([initial_costs[q] for q in v]) + 4*fixcosts
        sol1,f1 = combine(initial_routes[a],requests[b])
        sol2,f2 = combine(sol1,requests[c]) 
        sol3,f3 = combine(sol2,requests[d]) 
        new_costs = sol3[-1][0]
        val = new_costs + fixcosts 
        if val < costs:
            valid_vierer.append(v)
            vierer_cost.append(new_costs)
    vierer_cost.append(0)
    
    valid_funfer = []
    funfer_cost  = []
    for f in funfer:
        a,b,c,d,e = f
        costs = sum([initial_costs[q] for q in f]) + 5*fixcosts
        sol1,f1 = combine(initial_routes[a],requests[b])
        sol2,f2 = combine(sol1,requests[c]) 
        sol3,f3 = combine(sol2,requests[d]) 
        sol4,f4 = combine(sol3,requests[e]) 
        new_costs = sol4[-1][0]
        val = new_costs + fixcosts   
        if val < costs:
            valid_funfer.append(f)
            funfer_cost.append(new_costs)
    funfer_cost.append(0)
    valid_dreier = []
    dreier_cost = []
    
    for d in dreier:
        a,b,c = d
        costs = sum([initial_costs[q] for q in d]) + 3*fixcosts
        sol1,f1 = combine(initial_routes[a],requests[b])
        sol2,f2 = combine(sol1,requests[c])
        new_costs = sol2[-1][0]
        val = new_costs + fixcosts  
        if val < costs:
            valid_dreier.append(d)
            dreier_cost.append(new_costs)
    dreier_cost.append(0)
    
    valid_zweier = []
    zweier_cost = []
    for z in zweier:
        a,b = z
        costs = sum([initial_costs[q] for q in z]) + 2*fixcosts
        sol1,f1 = combine(initial_routes[a],requests[b])   
        new_costs = sol1[-1][0]
        val = new_costs + fixcosts  
        if val < costs:
            valid_zweier.append(z)
            zweier_cost.append(new_costs)
    zweier_cost.append(0)    
    
    combinations_vv = []
    vv_costs = []
    for k in range(len(valid_vierer)):
        v = valid_vierer[k]

        for j in range(k+1,len(valid_vierer)):
            v2 = valid_vierer[j]
            diff = set(v)-set(v2)

            if len(diff) == 4:
                # this combination is possible!
                costs = sum([initial_costs[q] for q in v2+v]) + 8*fixcosts
                val = vierer_cost[j] + vierer_cost[k] + 2*fixcosts   # obj val of combination

                if val < costs:
                    combinations_vv.append((k,j))
                    vv_costs.append(val)
    combinations_vvv = []

    for k in range(len(valid_vierer)):
        v = valid_vierer[k]

        for j in range(k+1,len(valid_vierer)):
            v2 = valid_vierer[j]

            for l in range(j+1,len(valid_vierer)):
                v3 = valid_vierer[l]
                union = set(v) | set(v2) | set(v3) 

                if len(union) == 12:
                    # this combination is possible!
                    costs = sum([initial_costs[q] for q in v2+v+v3]) + 12*fixcosts
                    val = vierer_cost[j] + vierer_cost[k] + vierer_cost[l] + 3*fixcosts   # obj val of combination

                    if val < costs:
                        combinations_vvv.append((k,j,l))
    combinations_vvvv = []

    for k in range(len(valid_vierer)):
        v = valid_vierer[k]

        for j in range(k+1,len(valid_vierer)):
            v2 = valid_vierer[j]

            for l in range(j+1,len(valid_vierer)):
                v3 = valid_vierer[l]
                
                for m in range(l+1,len(valid_vierer)):
                    v4 = valid_vierer[m]
                    union = set(v) | set(v2) | set(v3)  | set(v4) 

                    if len(union) == 16:
                        # this combination is possible!
                        costs = sum([initial_costs[q] for q in v2+v+v3+v4]) + 16*fixcosts
                        val = vierer_cost[j] + vierer_cost[k] + vierer_cost[l]+ vierer_cost[m] + 4*fixcosts   # obj val of combination
    
                        if val < costs:
                            combinations_vvv.append((k,j,l))

    combinations_dd = []
    dd_costs = [] 

    for k in range(len(valid_dreier)):
        d = valid_dreier[k]

        for j in range(k+1,len(valid_dreier)):
            d2 = valid_dreier[j]
            union = set(d) | set(d2)

            if len(union) == 6:
                # this combination is possible!
                costs = sum([initial_costs[q] for q in d+d2]) + 6*fixcosts
                val = dreier_cost[j] + dreier_cost[k] + 2*fixcosts   # obj val of combination

                if val < costs:
                    combinations_dd.append((k,j))
                    dd_costs.append(val)

    combinations_ddd = []
    ddd_costs = []
    for k in range(len(valid_dreier)):
        d = valid_dreier[k]

        for j in range(k+1,len(valid_dreier)):
            d2 = valid_dreier[j]

            for l in range(j+1,len(valid_dreier)):
                d3 = valid_dreier[l] 
                union = set(d) | set(d2) | set(d3)

                if len(union) == 9:
                    singles = list(set(all_indices) - union)
                    val = (dreier_cost[j] + dreier_cost[k] + dreier_cost[l] + 
                               + sum([initial_costs[q] for q in singles]) + 14*fixcosts)   # obj val of combination
                    combinations_ddd.append((k,j,l))
                    ddd_costs.append(val)

    combinations_dddd = []

    for k in range(len(valid_dreier)):
        d = valid_dreier[k]

        for j in range(k+1,len(valid_dreier)):
            d2 = valid_dreier[j]

            for l in range(j+1,len(valid_dreier)):
                d3 = valid_dreier[l] 

                for m in range(l+1,len(valid_dreier)):
                    d4 = valid_dreier[m] 
                    union = set(d) | set(d2) | set(d3) | set(d4)

                    if len(union) == 12:
                        costs = sum([initial_costs[q] for q in d+d2+d3+d4]) + 12*fixcosts
                        val = dreier_cost[j] + dreier_cost[k] + dreier_cost[l] + dreier_cost[m] + 4*fixcosts   # obj val of combination

                        combinations_dddd.append((k,j,l,m))

    combinations_5d = []
    val_5d = []

    for k in range(len(valid_dreier)):
        d = valid_dreier[k]

        for j in range(k+1,len(valid_dreier)):
            d2 = valid_dreier[j]

            for l in range(j+1,len(valid_dreier)):
                d3 = valid_dreier[l] 

                for m in range(l+1,len(valid_dreier)):
                    d4 = valid_dreier[m] 

                    for p in range(m+1,len(valid_dreier)):
                        d5 = valid_dreier[p]
                        union = set(d) | set(d2) | set(d3) | set(d4)| set(d5)

                        if len(union) == 15:
                            costs = sum([initial_costs[q] for q in d+d2+d3+d4+d5]) + 15*fixcosts
                            val = dreier_cost[j] + dreier_cost[k] + dreier_cost[l] + dreier_cost[m]+ dreier_cost[p] + 5*fixcosts   # obj val of combination

                            combinations_5d.append((k,j,l,m,p))
                            val_5d.append(val)

    combinations_6d = []

    for c in combinations_5d:
        i1,i2,i3,i4,i5 = c
        d1 = valid_dreier[i1]
        d2 = valid_dreier[i2]
        d3 = valid_dreier[i3]
        d4 = valid_dreier[i4]
        d5 = valid_dreier[i5]
        non_used_dreier = list(set(all_indices)-set(c))
        for o in non_used_dreier:
            d6 = valid_dreier[o]
            union = set(d1) | set(d2) | set(d3) | set(d4)| set(d5)| set(d6)

            if len(union) == 18:
                combinations_6d.append((i1,i2,i3,i4,i5,o))


    combinations_zz = []
    zz_vals = []

    for k in range(len(valid_zweier)):
        d = valid_zweier[k]

        for j in range(k+1,len(valid_zweier)):
            d2 = valid_zweier[j]
            union = set(d) | set(d2)

            if len(union) == 4:
                # this combination is possible!
                costs = sum([initial_costs[q] for q in d+d2]) + 4*fixcosts
                val = zweier_cost[j] + zweier_cost[k] + 2*fixcosts   # obj val of combination

                combinations_zz.append((k,j))
                zz_vals.append(val)

    combinations_zzz = []
    index=0
    #zz_vals[i] corresponds to index
    for k in range(len(valid_zweier)):
        d = valid_zweier[k] 
        for l in range(k+1,len(valid_zweier)):

            d2 = valid_zweier[l]
            for j in range(l+1,len(valid_zweier)):
                d3 = valid_zweier[j]

                union = set(d) | set(d2) | set(d3)

                if len(union) == 6:
                    combinations_zzz.append((k,l,j))


    combinations_4z = []
    z4_costs = []

    for k in range(len(valid_zweier)):
        d = valid_zweier[k]

        for j in range(k+1,len(valid_zweier)):
            d2 = valid_zweier[j]

            for l in range(j+1,len(valid_zweier)):
                d3 = valid_zweier[l]

                for m in range(l+1,len(valid_zweier)):
                    d4 = valid_zweier[m]
                    union = set(d) | set(d2) | set(d3) | set(d4)

                    if len(union) == 8:
                        # this combination is possible!
                        singles = list(set(all_indices) - union)
                        costs = base_costs+20*fixcosts
                        val = (zweier_cost[j] + zweier_cost[k] + zweier_cost[l] + zweier_cost[m]+ 
                               + sum([initial_costs[q] for q in singles]) + 16*fixcosts)   # obj val of combination

                        combinations_4z.append([k,j,l,m])
                        z4_costs.append(val)

    combinations_5z = []
    z5_costs = []

    for k in range(len(valid_zweier)):
        d = valid_zweier[k]

        for j in range(k+1,len(valid_zweier)):
            d2 = valid_zweier[j]

            for l in range(j+1,len(valid_zweier)):
                d3 = valid_zweier[l]

                for m in range(l+1,len(valid_zweier)):
                    d4 = valid_zweier[m]

                    for p in range(m+1,len(valid_zweier)):
                        d5 = valid_zweier[p]

                        union = set(d) | set(d2) | set(d3) | set(d4)| set(d5)

                        if len(union) == 10:
                            # this combination is possible!
                            singles = list(set(all_indices) - union)
                            costs = base_costs +20*fixcosts 
                            val = (zweier_cost[j] + zweier_cost[k] + zweier_cost[l] + zweier_cost[m]+zweier_cost[p]
                                + sum([initial_costs[q] for q in singles]) + 15*fixcosts)   # obj val of combination

                            combinations_5z.append([k,j,l,m,p])
                            z5_costs.append(val)

    combinations_6z = []
    z6_costs = []
    for k in range(len(valid_zweier)):
        d = valid_zweier[k]

        for j in range(k+1,len(valid_zweier)):
            d2 = valid_zweier[j]

            for l in range(j+1,len(valid_zweier)):
                d3 = valid_zweier[l]

                for m in range(l+1,len(valid_zweier)):
                    d4 = valid_zweier[m]

                    for p in range(m+1,len(valid_zweier)):
                        d5 = valid_zweier[p]

                        for o in range(p+1,len(valid_zweier)):
                            d6 = valid_zweier[o]
                            union = set(d) | set(d2) | set(d3) | set(d4)| set(d5)| set(d6)

                            if len(union) == 12:
                                singles = list(set(all_indices) - union)
                                combinations_6z.append([k,j,l,m,p,o])
                                val = (zweier_cost[j] + zweier_cost[k] + zweier_cost[l] + zweier_cost[m]
                                      + zweier_cost[p] + zweier_cost[o]+sum([initial_costs[q] for q in singles])
                                       + 9*fixcosts)   # obj val of combination

                                z6_costs.append(val)   

    combinations_7z = []
    z7_costs = []
    for k in range(len(valid_zweier)):
        d = valid_zweier[k]

        for j in range(k+1,len(valid_zweier)):
            d2 = valid_zweier[j]

            for l in range(j+1,len(valid_zweier)):
                d3 = valid_zweier[l]

                for m in range(l+1,len(valid_zweier)):
                    d4 = valid_zweier[m]

                    for p in range(m+1,len(valid_zweier)):
                        d5 = valid_zweier[p]

                        for o in range(p+1,len(valid_zweier)):
                            d6 = valid_zweier[o]

                            for r in range(o+1,len(valid_zweier)):
                                d7 = valid_zweier[r]
                                union = set(d) | set(d2) | set(d3) | set(d4)| set(d5)| set(d6)| set(d7)

                                if len(union) == 14:
                                    singles = list(set(all_indices)-union)
                                    val = (zweier_cost[j] + zweier_cost[k] + zweier_cost[l] + zweier_cost[m]
                                      + zweier_cost[p] + zweier_cost[o]+ zweier_cost[r]
                                           +sum([initial_costs[q] for q in singles])+ 8*fixcosts)
                                    z7_costs.append(val)
                                    combinations_7z.append([k,j,l,m,p,o,r])

    combinations_8z = []
    for c in combinations_7z:
        i1,i2,i3,i4,i5,i6,i7 = c
        d1 = valid_zweier[i1]
        d2 = valid_zweier[i2]
        d3 = valid_zweier[i3]
        d4 = valid_zweier[i4]
        d5 = valid_zweier[i5]
        d6 = valid_zweier[i6]
        d7 = valid_zweier[i7]
        non_used_zweier = list(set(all_indices)-set(d1+d2+d3+d4+d5+d6+d7))
        for o in range(len(valid_zweier)):
            d8 = valid_zweier[o]
            union = set(d1) | set(d2) | set(d3) | set(d4)| set(d5)| set(d6)| set(d7)| set(d8)

            if len(union) == 16:
                combinations_8z.append((i1,i2,i3,i4,i5,i6,i7,o))

    return(valid_zweier, valid_dreier, valid_vierer, valid_funfer,combinations_zz,combinations_zzz,
           combinations_4z,combinations_5z,combinations_6z,combinations_7z,combinations_8z,
           combinations_dd,combinations_ddd,combinations_dddd,combinations_5d,
           combinations_6d,combinations_vv,combinations_vvv,combinations_vvvv,
          zweier_cost,dreier_cost,vierer_cost,funfer_cost)

In [11]:
def best_combination(n,fixcosts,valid_zweier,valid_dreier,valid_vierer,valid_funfer,zweier_cost,dreier_cost,vierer_cost,funfer_cost):
    all_indices = range(n)
    initial_costs =[]
    base_costs = 0
    for s in initial_routes:
        c = s[-1][0]
        base_costs += c + fixcosts    
        initial_costs.append(c)
        
    vierer_cost.append(0)
    i = 0
    f_non_empty = 1
    v_non_empty = 1
    d_non_empty = 1
    zz_non_empty = 1
    zzz_non_empty = 1
    valid_dreier_plus = valid_dreier+[[]]    
    best_val = np.inf
    for f in valid_funfer+[[]]:
        j = 0
        nonempty = 2
        for v in valid_vierer+[[]]:
            k = 0
            union1 = set(f) | set(v) 
            if len(union1) == (len(f) + len(v)): # possible combi
                if len(f)==0:
                    f_c = 0
                    f_non_empty = 0
                else:
                    f_non_empty = 1
                    f_c = funfer_cost[i]
                if len(v)==0:
                    v_c = 0
                    v_non_empty = 0
                else:  
                    v_non_empty = 1
                    v_c = vierer_cost[j]
                filled = f_non_empty +v_non_empty
                singles = set(all_indices)-set(union1)
                sol_val = (f_c + v_c + sum([initial_costs[q] for q in singles]) + 
                            (len(singles) + filled) * fixcosts)
                if sol_val < best_val:
                    best_val = sol_val
                    best_combi = (f,v)
                
                valid_vierer_plus = valid_vierer+[[]]
                for ql in range(len(valid_vierer_plus)):
                    v2 = valid_vierer_plus[ql]
                    union11 = union1 | set(v2)
                    if len(union11) == len(union1) + len(v2):
                        if len(v2)==0:
                            v_c2 = 0
                            v_non_empty2 = 0
                        else: 
                            v_non_empty2 = 1 
                            v_c2 = vierer_cost[ql]                             
                                                  
                        for kk in range(len(valid_dreier_plus)):
                            l = 0
                            d = valid_dreier_plus[kk]
                            union2 = union11 | set(d)
                            if len(union2) == len(union11) + len(d):
                                if len(d)==0:
                                    d_c = 0
                                    d_non_empty = 0
                                else: 
                                    d_non_empty = 1
                                    d_c = dreier_cost[kk] 
                                singles2 = singles - set(d) - set(v2)

                                for qq in range(len(valid_dreier_plus)):
                                    d2 = valid_dreier_plus[qq]
                                    union33 = union2 | set(d2)
                                    if len(union33) == len(union2) + len(d2):
                                        if len(d2)==0:
                                            d_c2 = 0
                                            d_non_empty2 = 0
                                        else: 
                                            d_non_empty2 = 1
                                            d_c2 = dreier_cost[qq] 
                                        singles33 = singles2 - set(d2)
                                        
                                        for qll in range(len(valid_dreier_plus)):
                                            d3 = valid_dreier_plus[qll]
                                            union22 = union33 | set(d3)
                                            if len(union22) == len(union33) + len(d3):
                                                if len(d3)==0:
                                                    d_c3 = 0
                                                    d_non_empty3 = 0
                                                else: 
                                                    d_non_empty3 = 1
                                                    d_c3 = dreier_cost[qll] 
                                                singles22 = singles33 - set(d3)

                                                for qq5 in range(len(valid_dreier_plus)):
                                                    d4 = valid_dreier_plus[qq5]
                                                    union3 = union22 | set(d4)
                                                    if len(union3) == len(union22) + len(d4):
                                                        if len(d4)==0:
                                                            d_c4 = 0
                                                            d_non_empty4 = 0
                                                        else: 
                                                            d_non_empty4 = 1
                                                            d_c4 = dreier_cost[qq5] 
                                                        singles3 = singles22 - set(d4)
                                                        non_empt = filled + v_non_empty2 + d_non_empty + d_non_empty2+ d_non_empty3 + d_non_empty4
                                                        sol_val = (f_c + v_c +v_c2+ d_c +d_c2 + d_c3 +d_c4 + sum([initial_costs[q] for q in singles3]) + 
                                                                  (len(singles3)+ non_empt) * fixcosts)
                                                        if sol_val < best_val:
                                                            best_val = sol_val
                                                            best_combi = (f,v,v2,d,d2,d3,d4)

                                                        for pp in range(len(valid_zweier)):
                                                            z = valid_zweier[pp]
                                                            union4 = union3 |set(z) 
                                                            if len(union4) == len(union3) + len(z): # = possible combi
                                                                zweier_plus = valid_zweier+[[]]
                                                                for o in range(len(zweier_plus)):
                                                                    zz = zweier_plus[o]
                                                                    union5 = union4 | set(zz)
                                                                    if len(union5) == len(union4) + len(zz):
                                                                        if len(zz)==0:
                                                                            zz_non_empty = 0
                                                                            zz_costs = 0
                                                                        else:
                                                                            zz_non_empty = 1
                                                                            zz_costs = zweier_cost[o]
                                                                        for q in range(len(zweier_plus)):
                                                                            zzz = zweier_plus[q]
                                                                            union6 = union5 | set(zzz)
                                                                            if len(union6) == len(union5) + len(zzz):
                                                                                if len(zzz)==0:
                                                                                    zzz_non_empty = 0 
                                                                                    zzz_costs = 0
                                                                                else:
                                                                                    zzz_non_empty = 1
                                                                                    zzz_costs = zweier_cost[q]

                                                                                for oo in range(len(zweier_plus)):
                                                                                    z4 = zweier_plus[oo]
                                                                                    union7 = union6 | set(z4)
                                                                                    if len(union7) == len(union6) + len(z4):
                                                                                        if len(z4)==0:
                                                                                            z4_non_empty = 0
                                                                                            z4_costs = 0
                                                                                        else:
                                                                                            z4_non_empty = 1
                                                                                            z4_costs = zweier_cost[oo]
                                                                                        for ooo in range(len(zweier_plus)):
                                                                                            z5 = zweier_plus[ooo]
                                                                                            union8 = union7 | set(z5)
                                                                                            if len(union8) == len(union7) + len(z5):
                                                                               
                                                                                                if len(z5)==0:
                                                                                                    z5_non_empty = 0 
                                                                                                    z5_costs = 0
                                                                                                else:
                                                                                                    z5_non_empty = 1
                                                                                                    z5_costs = zweier_cost[ooo]
                                                                                                for o4 in range(len(zweier_plus)):
                                                                                                    z6 = zweier_plus[o4]
                                                                                                    union9 = union8 | set(z6)
                                                                                                    if len(union9) == len(union8) + len(z6):
                                                                                                        if len(z6)==0:
                                                                                                            z6_non_empty = 0 
                                                                                                            z6_costs = 0
                                                                                                        else:
                                                                                                            z6_non_empty = 1
                                                                                                            z6_costs = zweier_cost[o4]

                                                                                                        non_empty = (filled + zzz_non_empty +zz_non_empty +1+d_non_empty
                                                                                                                     +d_non_empty2+d_non_empty3 + v_non_empty2
                                                                                                                     +d_non_empty4+z4_non_empty +z5_non_empty +z6_non_empty )
                                                                                                        singles4 = singles3 - set(z)- set(zz)- set(zzz)- set(z4)- set(z5)- set(z6)                                                                                 
                                                                                                        sol_val = (f_c + v_c +v_c + d_c +d_c2+ d_c3 +d_c4+ zweier_cost[pp]+ zz_costs +zzz_costs
                                                                                                                   +z4_costs+z5_costs+z6_costs+
                                                                                                                   sum([initial_costs[qn] for qn in singles4])+
                                                                                                                           (len(singles4) + non_empty) * fixcosts)
                                                                                                        if sol_val < best_val:
                                                                                                            best_val = sol_val
                                                                                                            best_combi = (f,v,v2,d,d2,d3,d4,z,zz,zzz,z4,z5,z6) 
                                 

            j += 1
        i += 1
        
    return(best_val, best_combi)